In [ ]:
import os
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
from eyemouse import *

%matplotlib inline

In [ ]:
e_list = [load_eyetracker_file(e) for e in eyet_files]     # lists of dataframes containing respective data
m_list = [load_mouse_file(m) for m in mous_files]
g_list = [load_game_file(g) for g in mous_files]

t = [
    make_tidy(e_list[i], m_list[i], g_list[i]) 
    for i in range(len(e_list))
]                                                          # create tidy dfs per subject along timestamp index

mpivs = []                                                 # holding lists for pivoted histograms
epivs = []

for i in range(len(m_list)):                               # for each subject
    
    # MOUSE HISTOGRAMS
    m_list[i], mpvt = mouse_hist(m_list[i], g_list[i])
    mpivs.append(mpvt)
    
    # EYE HISTOGRAMS
    
    e_list[i], epvt = eye_hist(e_list[i], g_list[i])
    epivs.append(epvt)

# Export

In [ ]:
for i in range(len(mpivs)):
    mp = mpivs[i]
    ep = epivs[i]
    
    mp[list(range(36)) + [999, 'bp', 'wp', 'zet']].to_csv(os.path.join(output_dir, 'mouse {}.csv'.format(i)))
    ep[list(range(36)) + [999, 'bp', 'wp', 'zet']].to_csv(os.path.join(output_dir, 'eye {}.csv'.format(i)))

# Histograms

In [ ]:
mp = mpivs[0][[i for i in mpivs[0].columns if i in list(range(36))]]
mpn = mp / mp.sum(axis=0)
mpn.head()

In [ ]:
ep = epivs[0][[i for i in range(36)]]
epn = ep / ep.sum(axis=0)
epn.head()

# Statistics

In [ ]:
figrows = len(e_list)                                       # n rows == n subjects
fig, axes = plt.subplots(figrows, 2, figsize=(10, 40))   # initialize figure
scatterkws = {'linestyle': 'none', 'marker': 'o', 'markersize': 1, 'alpha': .1}
                                                            # set kws for scatter plot
xrs = []                                                    # initialize lists for x and y r values (convert to numpy?)
yrs = []
for subject, ee in enumerate(e_list):                       # for every subject
    for j, dim in enumerate(['x', 'y']):                    # for x and y dimensions      
        ul = 4 if dim=='y' else 9
        t_ = t[subject]                                     # select subject data
        nanfilter = pd.notnull(t_['eye' + dim]) & pd.notnull(t_['mou' + dim])
                                                            # filter records without mouse and eye vals
        trialfilter = t[subject]['task']=='playing'         # filter convenience records
        tt = t[subject].loc[nanfilter & trialfilter]        # apply filters
        eyebounds = (tt['eye' + dim] >= 0) & (tt['eye' + dim] <= ul)
        mousebounds = (tt['mou' + dim] >= 0) & (tt['mou' + dim] <= ul)
                                                            # filter eye  and mousecoordinates off board
        tt = tt.loc[eyebounds & mousebounds]                # apply eyebounds and mousebounds filters
        lr = stats.linregress(tt['eye' + dim], tt['mou' + dim])
                                                            # regress eye coordinate against mouse coordinate
        
        if dim == 'x':                                        # save regression coefficients
            xrs.append(lr.rvalue)
        else:
            yrs.append(lr.rvalue)
             
        ax = axes[subject, j]                               # select axes
        linex = [-2, 11]
        liney = [-2*lr.slope + lr.intercept, 11*lr.slope + lr.intercept]
                                                            # x and y coordinates for plotting regression line
        ax.plot(linex, liney, color='black')                # plot regression line
        ax.plot(tt['eye' + dim], tt['mou' + dim], **scatterkws)
                                                            # scatter eye coordinates against mouse coordinates

        
plt.setp(axes[:, 0], xlim=[-1, 10], ylim=[-1, 10])          # plot formatting
plt.setp(axes[:, 1], xlim=[-1, 5], ylim=[-1, 5])
plt.setp(axes[:-1, :], xticklabels=[])
plt.setp(axes[-1, 0], xlabel='Eye Pos (board coord.)')
plt.setp(axes[4, 0], ylabel='Mouse Pos (board coord.)')
plt.setp(axes[0, 0], title='X')
plt.setp(axes[0, 1], title='Y')

sns.despine()

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(5, 5))
scatterkws['markersize'] = 10
scatterkws['alpha'] = .9
axes.plot(xrs, yrs, **scatterkws)
plt.setp(axes, xlabel='X dim Pearson R', ylabel='Y dim Pearson R', xlim=[0, 1], ylim=[0, 1])

sns.despine()
plt.tight_layout()

## Old stuff from debugging

In [ ]:
_t = m.loc[m['turn'].isin([117,118,119])]
_t

In [ ]:
g.loc[g.status.isin(['playing', 'draw', 'win'])].sort_index().head(30)

In [ ]:
gt = g.loc[g.status.isin(['playing', 'draw', 'win'])&g['is human']==1].set_index('turn')
mpvt.loc[gt.index, 'true rt'] = gt['rt']
mpvt['rt'] - mpvt['true rt']

In [ ]:
g = g_list[0]
g = g.loc[g['status']=='playing']
m = m_list[0]

probe = g.loc[(g['gi']==1)&(g['mi']==2), :]

endtime = probe.index.values[0]
starttime = endtime - probe['rt'].values[0]
print(endtime, starttime)
mp = m.loc[(m.index>=starttime)&(m.index<=endtime), ['x', 'y']].astype(float)
mp['tstemp'] = mp.index
mp['dur'] = -mp['tstemp'].diff(periods=-1)
mp['dur'].sum()

In [ ]:
# mt = np.array([str(starttime)] + probe['mt'].values[0].split(',') + [str(endtime)]).astype(float)
mt = np.array(probe['mt'].values[0].split(',')).astype(float)
mt = mt[mt >= starttime]
(mt[1:] - mt[:-1]).sum()

In [ ]:
probe['rt']